In [1]:
import pandas as pd 
import numpy as np
import pickle

In [31]:
df = pd.read_csv("full_data_out_rem.csv")
df.head()

,type,total_price,carat,price_per_carat,color,shape,length,width,height,clarity,cut,color_intensity,origin,treatment,cut_quality
0,Blue Sapphire,501,0.91,551.0,Blue,Oval,6.76,4.74,3.24,Slightly Included,Mixed Brilliant,Very Light,Ceylon (Sri Lanka),No Enhancement,Very Good
1,Blue Sapphire,501,0.77,651.0,Blue,Round,5.19,5.02,3.77,Very Slightly Included,Brilliant,Vivid,Ceylon (Sri Lanka),Heated,Fair
2,Pink Sapphire,501,0.91,551.0,Pinkish Purple,Radiant,6.40,4.27,3.48,Very Slightly Included,Radiant Cut,Intense,Ceylon (Sri Lanka),No Enhancement,Good
3,Pink Sapphire,501,0.97,516.0,Pinkish Purple,Radiant,5.78,4.57,3.85,Slightly Included,Radiant Cut,Medium,Ceylon (Sri Lanka),No Enhancement,Good
4,Pink Sapphire,501,0.97,516.0,Purplish Pink,Round,6.44,6.35,3.28,Very Slightly Included,Mixed Brilliant,Intense,Ceylon (Sri Lanka),No Enhancement,Fair


In [32]:
# Split data into training and testing data
from sklearn.model_selection import train_test_split

data = df.drop(['length', 'width', 'height'], axis='columns')
X = df[['carat','price_per_carat', 'color', 'shape', 'clarity', 'cut', 'color_intensity', 'origin', 'treatment','cut_quality','type']]
y = df['total_price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [33]:
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Poor', 'Fair', 'Good','Very Good','Excellent']])
X_train['cut_quality_encoded'] = oe.fit_transform(X_train[['cut_quality']])
X_test['cut_quality_encoded'] = oe.transform(X_test[['cut_quality']])

# Handle target
type_mean = X_train.groupby('type')['price_per_carat'].mean()
shape_means = X_train.groupby('shape')['price_per_carat'].mean()
origin_means = X_train.groupby('origin')['price_per_carat'].mean()
color_means = X_train.groupby('color')['price_per_carat'].mean()
color_intensity_means = X_train.groupby('color_intensity')['price_per_carat'].mean()
clarity_means = X_train.groupby('clarity')['price_per_carat'].mean()
treatment_means = X_train.groupby('treatment')['price_per_carat'].mean()
cut_means = X_train.groupby('cut')['price_per_carat'].mean()

# Map target encoding to the training data
X_train['shape_encoded'] = X_train['shape'].map(shape_means)
X_train['origin_encoded'] = X_train['origin'].map(origin_means)
X_train['color_encoded'] = X_train['color'].map(color_means)
X_train['color_intensity_encoded'] = X_train['color_intensity'].map(color_intensity_means)
X_train['clarity_encoded'] = X_train['clarity'].map(clarity_means)
X_train['cut_encoded'] = X_train['cut'].map(cut_means)
X_train['treatment_encoded'] = X_train['treatment'].map(treatment_means)
X_train['type_encoded'] = X_train['type'].map(type_mean)

# Map target encoding to the test data
# Handle unseen categories by filling with global mean
global_mean = X_train['price_per_carat'].mean()
X_test['shape_encoded'] = X_test['shape'].map(shape_means).fillna(global_mean)
X_test['origin_encoded'] = X_test['origin'].map(origin_means).fillna(global_mean)
X_test['color_encoded'] = X_test['color'].map(color_means).fillna(global_mean)
X_test['color_intensity_encoded'] = X_test['color_intensity'].map(color_intensity_means).fillna(global_mean)
X_test['clarity_encoded'] = X_test['clarity'].map(clarity_means).fillna(global_mean)
X_test['cut_encoded'] = X_test['cut'].map(cut_means).fillna(global_mean)
X_test['treatment_encoded'] = X_test['treatment'].map(treatment_means).fillna(global_mean)
X_test['type_encoded'] = X_test['type'].map(type_mean).fillna(global_mean)

X_train = X_train.drop(columns=['price_per_carat', 'color', 'shape', 'color_intensity', 'origin', 'cut', 'treatment', 'clarity','cut_quality','type'])
X_test = X_test.drop(columns=['price_per_carat', 'color', 'shape', 'color_intensity', 'origin', 'cut', 'treatment', 'clarity','cut_quality','type'])
print(X_train.dtypes)

carat                      float64
cut_quality_encoded        float64
shape_encoded              float64
origin_encoded             float64
color_encoded              float64
color_intensity_encoded    float64
clarity_encoded            float64
cut_encoded                float64
treatment_encoded          float64
type_encoded               float64
dtype: object


In [34]:
# OrdinalEncoder mapping
cut_quality_mapping = {category: idx for idx, category in enumerate(oe.categories_[0])}
print(cut_quality_mapping)

# Store target encoding mappings
shape_mapping = shape_means.to_dict()
origin_mapping = origin_means.to_dict()
color_mapping = color_means.to_dict()
color_intensity_mapping = color_intensity_means.to_dict()
clarity_mapping = clarity_means.to_dict()
treatment_mapping = treatment_means.to_dict()
cut_mapping = cut_means.to_dict()
type_mapping = type_mean.to_dict()

# Print one mapping as an example
print("Shape Mapping:", shape_mapping)

# Store global mean
global_mean_value = global_mean
print("Global Mean:", global_mean_value)

# Combine all mappings into a dictionary
all_mappings = {
    "cut_quality_mapping": cut_quality_mapping,
    "shape_mapping": shape_mapping,
    "origin_mapping": origin_mapping,
    "color_mapping": color_mapping,
    "color_intensity_mapping": color_intensity_mapping,
    "clarity_mapping": clarity_mapping,
    "treatment_mapping": treatment_mapping,
    "cut_mapping": cut_mapping,
    "type_mapping": type_mapping,
    "global_mean": global_mean_value,
}

# Save to a pickle file
with open("mappings.pkl", "wb") as f:
    pickle.dump(all_mappings, f)


{'Poor': 0, 'Fair': 1, 'Good': 2, 'Very Good': 3, 'Excellent': 4}
Shape Mapping: {'Asscher - Octagon': 2007.3068137254904, 'Cushion': 2946.7351812080537, 'Emerald Cut': 2346.2185017421602, 'Fancy': 1395.71875, 'Heart': 3277.0569620253164, 'Marquise': 1970.7735849056603, 'Oval': 2973.14115430622, 'Pear': 2535.1600833333337, 'Princess': 1574.2571428571428, 'Radiant': 1583.3932116788321, 'Round': 2494.715630769231, 'Trillion': 1191.3628318584072}
Global Mean: 2636.6688047028088


In [35]:
from xgboost import XGBRegressor
import xgboost
from sklearn.metrics import *
model = XGBRegressor(
    n_estimators=20,
    random_state=42,
    learning_rate=0.4
    )

model.fit(X_train, y_train)
y_predict = model.predict(X_test)
y_predict_train = model.predict(X_train)

# Predicting the accuracy score for Gradient Booster
score=r2_score(y_test, y_predict)
score_train = r2_score(y_train, y_predict_train)
print('-------Gradient Booster-------')
print('r2_score is', score)
print('r2_score for training is', score_train)
print('mean_squared_error is==', mean_squared_error(y_test,y_predict))
print('root_mean_squared_error is==',np.sqrt(mean_squared_error(y_test, y_predict)))

-------Gradient Booster-------
r2_score is 0.9012661122386352
r2_score for training is 0.965839687234875
mean_squared_error is== 19534923.165922984
root_mean_squared_error is== 4419.832934164252


In [36]:
# Save the trained model to a pickle file
with open("xgb_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully!")


Model saved successfully!
